# Compare unets with multiple inputs
Vanilla u net  
Unet with 2 inputs  
Unet with 2 inputs on different depth  
Double U net  
Triple U net  

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/M_DV_V2022

Mounted at /content/gdrive
/content/gdrive/MyDrive/M_DV_V2022


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score

from tensorflow.keras.layers import Input 
from keras.utils.vis_utils import plot_model

from unet_detection.models.vanilla_unet import vanilla_unet
from unet_detection.models.unet_2input import unet_2input
from unet_detection.models.unet_2input_deep import unet_2input_deep
from unet_detection.models.double_unet import double_unet
from unet_detection.models.triple_unet import triple_unet

### All nets

In [3]:
seed = 123

vUnet = vanilla_unet()
unet2i = unet_2input()
unet2i_d = unet_2input_deep()
dUnet = double_unet()
tUnet = triple_unet()

### Import data set and show

In [4]:
X_train = np.load("data/u_net_roofs/X_data.npy") 
X_70_train = np.load("data/u_net_roofs/X_70_data.npy")
y_train = np.load("data/u_net_roofs/y_data.npy")

X_val = np.load("data/u_net_roofs/X_data_val.npy")
X_70_val = np.load("data/u_net_roofs/X_70_data_val.npy")
y_val = np.load("data/u_net_roofs/y_data_val.npy")

X_test = np.load("data/u_net_roofs/X_data_test.npy")
X_70_test = np.load("data/u_net_roofs/X_70_data_test.npy")
y_test = np.load("data/u_net_roofs/y_data_test.npy")

In [5]:
def redesign_y(y):
  n,r1,c1,d = y.shape
  # Adds a new dimension of layer too have two class problem.
  yy = np.append(y, np.zeros((n, r1, c1,d)), axis=3)
  for i in range(int(y.max()-1)):  
    yy = np.append(yy, np.zeros((n, r1, c1,d)), axis=3)
  yy1 = yy.copy()
  yy1[:,:,:,0] = 0 # reset map
  for i in range(n):
    values = yy[i,:,:,0]
    for r in range(r1):
      for c in range(c1):
        value = yy[i,r,c,0]
        yy1[i,r,c,int(value)] = 1

  return yy1

In [6]:
y_train = redesign_y(y_train)
y_val = redesign_y(y_val)
y_test = redesign_y(y_test)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(100, 128, 128, 2)
(20, 128, 128, 2)
(50, 128, 128, 2)


In [ ]:
# Scale image down so each border is the same size when upsizing!
X_70_train = X_70_train[:,4:-4,4:-4,:]
X_70_val = X_70_val[:,4:-4,4:-4,:]
X_70_test = X_70_test[:,4:-4,4:-4,:]

## Set up net

In [ ]:
input_img1 = Input(shape=(128,128,177))
input_img2 = Input(shape=(56,56,220))

model = dUnet.get_unet(input_img1, input_img2, n_classes=2, last_activation='softmax')

model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

## Run test

In [7]:
n = 10
epochs = 100
f1_scores = []
jacards = []

input_img1 = Input(shape=(128,128,177))
input_img2 = Input(shape=(56,56,220))

for i in range(n):
  # Build model
  model = None
  model = vUnet.get_unet(input_img1,
                         n_classes=2, last_activation='softmax')

  model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
  
  # Run net
  hh = model.fit([X_train],
                  y_train, 
                  batch_size=16,
                  epochs=epochs,
                  verbose=0)
  
  # Save scores
  pred = model.predict([X_test])
  f1 = f1_score(y_test.argmax(axis=3).flatten(), pred.argmax(axis=3).flatten())
  f1_scores

  print(f"Round {i+1} of {n}. F1 score: {f1}")


f1_scores = np.array(f1_scores)
  

Round 1 of 10. F1 score: 0.6092769257931169
Round 2 of 10. F1 score: 0.7461402325149354
Round 3 of 10. F1 score: 0.7158214912210116
Round 4 of 10. F1 score: 0.7993460886413175
Round 5 of 10. F1 score: 0.5616651623181909
Round 6 of 10. F1 score: 0.7082598966275225
Round 7 of 10. F1 score: 0.6289953353195273
Round 8 of 10. F1 score: 0.7767192198521766
Round 9 of 10. F1 score: 0.44540359466597956
Round 10 of 10. F1 score: 0.2949445796161124


## Display model net